# Generator V2

Ferramenta que pega no conteudo das noticias (em formato json) e gera posts.

O tweet tem os seguintes elementos: 
* Alegação;
* Classificação;
* Hashtags relevantes;
* Emojis;
* URL;

### Os elementos do tweet TEM aletoriedade:
Baseia-se nas seguintes regras:

        1) Os 5 elemntos do twitter devem estar presentes;
        2) Os elemntos são adicionados a uma lista.
        3) a lista é ordenada por uma ordem random. O post é criado 
        


In [2]:
import os
import re
import json
import pandas as pd
import random
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
from keybert import KeyBERT
import emoji #https://carpedm20.github.io/emoji/

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mariabarbosa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mariabarbosa/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/opt/homebrew/anaconda3/envs/py3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Gerador

In [4]:
#pip install import-ipynb
import hashtags_generetor
import emojis
import verify_post


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mariabarbosa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
def generate_capslock_allegation(data):
    
    text = re.sub(r'[^\w\s]', '', data['allegation']) 
    words_allegation = [word.lower() for word in text.strip('\n').split() ]
    
    
    ## Definir o modelo
    kw_model = KeyBERT()
    words_bert = [word for word, value in kw_model.extract_keywords(docs=text, keyphrase_ngram_range=(1,1))]
                                        
    allegation =  []
    
    for word in text.strip('\n').split():
        if word.lower() in words_bert:
            allegation.append(word.upper())
        else:
            allegation.append(word)
    
    allegation = ' '.join(allegation)
    post = []

    #emoji.emojize(':thumbs_up:')
    post.append(allegation)
    #+ choice(dic[data['evaluation']])
    return post

In [38]:
def main():
    posts = {}
    ##### Read the json files with de news
    for file in os.listdir("../../extractors/Snopes/extractions"):
        with open("../../extractors/Snopes/extractions/" + file) as f:
            data = json.load(f)
            f.close()
            
        post = generate_capslock_allegation(data)
      #  if (verify_post.verify_length(' '.join (post)) == False):
      #      post = post[:280]

        post.append( emoji.emojize(':red_exclamation_mark:') + data['evaluation'].upper() + emoji.emojize(':red_exclamation_mark:'))

        
        '''
        Introduzir aleatoriedade;
        '''
       
        hashtags = hashtags_generetor.generateHashtags(' '.join(data['postText']).split('\n'))
        hashtags = [' #'+h for h in hashtags]
        post.append ( ' '.join (hashtags))
        post.append(data['url'])
        post.append (emojis.emojis_selector(data['evaluation']))
        
        
        seed = random.randint(0,5)
        if verify_post.verify_length(' '.join (post)):
          post = post[:280]

        
        while (verify_post.verify_length(' '.join (post)) and seed > 0):
                seed = seed - 1
                post.append(emojis.emojis_selector(data['evaluation']))


        random.shuffle(post) 

        post = ' '.join(post)
        
        print(post)



        df = pd.read_csv('snopes_posts_v2.csv')

        row = {'id': data['id'], 'post': post}

        # Adicionar a nova linha ao dataset
        df = df.append(row,  ignore_index=True)

        #Retirar colunaas que estão a mais
        df = df[['id','post' ]]


        df.to_csv('snopes_posts_v2.csv')
            
        print('--------------------------------------------')
        f.close()
        
        

        
        

In [39]:
main()

😭 https://www.snopes.com/fact-check/valentines-day-invented-greeting-card-companies/  #holiday VALENTINES DAY was INVENTED by GREETING CARD companies 😫 😞 😫 😡 ❗FALSE❗
--------------------------------------------
❗MISCAPTIONED❗ A VIDEO shows TRUCKERS from South CAROLINA on their way to join a FREEDOM CONVOY in Canada 😔 https://www.snopes.com/fact-check/freedom-convoy-canada/  #truck
--------------------------------------------
❗FALSE❗ 🔫 😔 😫 https://www.snopes.com/fact-check/unique-tip-automatic-headlights/ 🔫 🔫  #automatic An online ADVERTISEMENT revealed an unusual or unique tip or TRICK for CARS with AUTOMATIC HEADLIGHTS
--------------------------------------------
https://www.snopes.com/fact-check/putin-bombs-biden-owned-villa/ 😫 😩 😩 ❗FALSE❗  #real 💀 😔 RUSSIA BOMBED a Bidenowned VILLA and several US BIOLABS in UKRAINE
--------------------------------------------
 #president In September 2021 an initial news release announcing US Rep LAUREN BOEBERTS IMPEACHMENT articles again President 

KeyboardInterrupt: 